# 转置卷积

卷积要么宽高不变，要么宽高减小，**但是像素级预测需要最后换高增大**

- 转置卷积则可以用来增大输入高宽

$$Y[i:i+h, j:j+w]+=x[i,j]\cdot\text{kernel}$$

- padding作用在输出上
- 转置卷积也是卷积
    - 相当于上采样，将输入和核进行了重新排列

## 重新排列

- 转置卷积相当于只是改变了高宽
- 填充：p，步幅：s
    - 在行和列之间插入s-1行或列
    - 输入的上下左右填充 k-p-1
    - 核矩阵：上下左右反转
    - 然后正常卷积（填充为0、步幅为1）

$$
\begin{cases}
n'\geq s\times n + k - 2p - s \\
k = 2p+s
\end{cases}
$$

## 转置

- 对于卷积$Y=X\star W$
    - 可以对 W 构造出一个 V，使得卷积等价于$Y=V\cdot X$
- 而转置卷积则等价于$Y'=V^T\cdot X'$

> 转置卷积是卷积，但不是反卷积

In [3]:
import torch
from torch import nn
from d2l import torch as d2l

In [4]:
""" No padding Stride = 1 """
def trans_conv(X, k):
    h, w = k.shape
    Y = torch.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Y[i:i+h, j:j+w] += X[i, j] * k
    return Y

In [5]:
X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
k = X.clone()
trans_conv(X, k)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

In [6]:
X, k = X.reshape(1, 1, 2, 2), k.reshape(1, 1, 2, 2)
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)
tconv.weight.data=k
tconv(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<SlowConvTranspose2DBackward0>)

In [8]:
# 填充 相当于遮罩输出
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, padding=1, bias=False)
tconv.weight.data = k
tconv(X)

tensor([[[[4.]]]], grad_fn=<SlowConvTranspose2DBackward0>)

In [10]:
# (img_h + kernel_h - stride) / stride
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, bias=False)
tconv.weight.data = k
tconv(X)

tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<SlowConvTranspose2DBackward0>)